# Notebook 02: Building a Simple Agent with Tools

## 🎯 What is This Notebook About?

In this notebook, we'll build autonomous agents with tools using LlamaStack. We'll start simple and build up to custom tools.

**What we'll learn:**
1. How to create an agent with built-in tools (web_search)
2. How to create sessions and execute turns
3. How to see tool calls using AgentEventLogger
4. How to create custom (client-side) tools
5. How to use custom tools with agents

---

## ⚙️ Prerequisites

- LlamaStack server running (see Module README)
- Ollama running with llama3.2:3b model
- Python environment with dependencies installed
- TAVILY_SEARCH_API_KEY (for web_search tool)

In [ ]:
# Install required packages if not already installed
# This ensures the packages are available in the Jupyter kernel

# Check and install llama-stack-client>=0.3.1 and its dependencies
try:
    import llama_stack_client
    from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger
    print('✅ llama-stack-client>=0.3.1 already installed')
except (ImportError, ModuleNotFoundError) as e:
    # Note: Server version 0.3.1 requires client >= 0.3.1
    print(f'Installing llama-stack-client>=0.3.1 and dependencies... (Error: {e})')
    print(f'Using Python: {__import__("sys").executable}')
    # Install llama-stack-client>=0.3.1 (this should install all dependencies)
    !pip install -q llama-stack-client>=0.3.1
    # Also install 'fire' explicitly as it's sometimes missing
    !pip install -q fire
    # Force reload by clearing module cache
    import sys
    if 'llama_stack_client' in sys.modules:
        del sys.modules['llama_stack_client']
    # Now import
    from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger
    print('✅ llama-stack-client>=0.3.1 installed and imported successfully')

# Check and install termcolor
try:
    from termcolor import cprint
    print('✅ termcolor already installed')
except ImportError:
    print('Installing termcolor...')
    !pip install -q termcolor
    from termcolor import cprint
    print('✅ termcolor installed and imported successfully')

# Import required libraries
import os
from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger
from termcolor import cprint

# Configuration
llamastack_url = os.getenv("LLAMA_STACK_URL", "http://localhost:8321")
model = os.getenv("LLAMA_MODEL", "ollama/llama3.2:3b")

print(f"📡 LlamaStack URL: {llamastack_url}")
print(f"🤖 Model: {model}")

# Initialize LlamaStack client
client = LlamaStackClient(base_url=llamastack_url)

# Verify connection
try:
    models = client.models.list()
    print(f"\n✅ Connected to LlamaStack")
    print(f"   Available models: {len(models)}")
except Exception as e:
    print(f"\n❌ Cannot connect to LlamaStack: {e}")
    print("   Please ensure LlamaStack is running:")
    print("   python scripts/start_llama_stack.py")
    raise


---

## Part 1: Create an Agent with Simple web_search

Let's start by creating an agent with a built-in tool: `web_search`.

**Built-in tools** are server-side tools provided by LlamaStack. They're ready to use - you just need to configure API keys.

In [ ]:
# Step 1: Create an agent with web_search tool
print("=" * 60)
print("Creating Agent with web_search Tool")
print("=" * 60)

agent = Agent(
    client,
    model=model,
    instructions="You are a helpful assistant. Use websearch tool to help answer questions.",
    tools=[
        {"type": "web_search"}
    ],
)

print("\n✅ Agent created successfully!")
print(f"   Tool: web_search (built-in)")
print(f"   Model: {model}")

In [ ]:
# Step 2: Create a session
# Sessions maintain conversation context
print("\n" + "=" * 60)
print("Creating Session")
print("=" * 60)

session_id = agent.create_session("test-session")
print(f"\n✅ Session created!")
print(f"   Session ID: {session_id}")
print(f"\n💡 Sessions maintain conversation history and context.")

In [ ]:
# Step 3: Execute a turn (give the agent a task)
# A turn is one interaction with the agent
print("\n" + "=" * 60)
print("Executing Turn with Task")
print("=" * 60)

task = "Which teams played in the NBA western conference finals of 2024"

print(f"\n📋 Task: {task}")
print(f"\n🔄 Agent execution (using AgentEventLogger):")
print("-" * 60)

cprint(f"User> {task}", "green")

# Create turn - this returns a response object
response = agent.create_turn(
    messages=[
        {
            "role": "user",
            "content": task,
        }
    ],
    session_id=session_id,
)

# AgentEventLogger shows tool calls visually!
# 🤔 = Agent reasoning
# 🔧 = Tool being executed
for log in AgentEventLogger().log(response):
    print(log, end="", flush=True)

print(f"\n\n{'=' * 60}")
print("✅ Turn completed!")
print(f"{'=' * 60}")

---

## Part 2: Create a Custom Tool

Now let's create **custom tools** - Python functions that run client-side.

**Key Points:**
- Custom tools are Python functions with **docstrings**
- Docstrings describe the tool to the LLM
- Tools are passed directly to the Agent
- Agent automatically detects and uses them

In [ ]:
# Step 1: Create a simple simulated IT environment
class SimpleITEnvironment:
    """Simple simulated IT environment for our custom tools."""
    def __init__(self):
        self.services = {
            "web-server": {"status": "online", "cpu": 45, "memory": 60},
            "database": {"status": "online", "cpu": 30, "memory": 50},
            "cache-service": {"status": "degraded", "cpu": 85, "memory": 90},
        }
    
    def get_service_status(self, service_name: str) -> dict:
        """Get service status."""
        return self.services.get(service_name, {"status": "not_found"})
    
    def restart_service(self, service_name: str) -> str:
        """Restart a service."""
        if service_name in self.services:
            self.services[service_name]["status"] = "online"
            self.services[service_name]["cpu"] = 20
            self.services[service_name]["memory"] = 30
            return f"Service {service_name} restarted successfully"
        return f"Service {service_name} not found"


# Initialize environment
env = SimpleITEnvironment()
print("✅ Created simulated IT environment")
print(f"   Services: {list(env.services.keys())}")

In [ ]:
# Step 2: Define custom tools as Python functions with docstrings
# The docstrings are CRITICAL - the LLM uses them to understand the tools!

def check_service_status(service_name: str) -> str:
    """
    Check the status of an IT service.
    
    Returns information about service health including CPU usage, memory usage, and current status.
    Use this to monitor service health and detect issues.
    
    :param service_name: The name of the service to check (e.g., 'web-server', 'database', 'cache-service')
    :return: A string describing the service status, CPU usage, and memory usage
    """
    status = env.get_service_status(service_name)
    if status.get("status") == "not_found":
        return f"Service '{service_name}' not found"
    
    return (
        f"Service: {service_name}\n"
        f"Status: {status['status']}\n"
        f"CPU Usage: {status['cpu']}%\n"
        f"Memory Usage: {status['memory']}%"
    )


def restart_service(service_name: str) -> str:
    """
    Restart an IT service that is not working properly.
    
    Use this when a service has failed or is in a degraded state.
    This will stop and start the service, which may cause brief downtime.
    
    :param service_name: The name of the service to restart (e.g., 'web-server', 'database')
    :return: A string confirming the restart operation
    """
    return env.restart_service(service_name)


def get_all_services() -> str:
    """
    Get the status of all services in the environment.
    
    Use this to get an overview of the entire system health.
    Returns a list of all services with their current status and metrics.
    
    :return: A string listing all services and their status
    """
    result = "All Services Status:\n"
    for name, status in env.services.items():
        result += f"\n{name}:\n"
        result += f"  Status: {status['status']}\n"
        result += f"  CPU: {status['cpu']}%\n"
        result += f"  Memory: {status['memory']}%\n"
    return result


print("\n✅ Custom tools defined:")
print("   - check_service_status(service_name: str)")
print("   - restart_service(service_name: str)")
print("   - get_all_services()")
print("\n💡 Key Points:")
print("   - Tools are Python functions with docstrings")
print("   - Docstrings describe what the tool does")
print("   - :param describes parameters")
print("   - The LLM reads these docstrings to understand when to use each tool")

In [ ]:
# Step 3: Create an agent with custom tools
# Pass the functions directly - Agent will detect them via docstrings!
print("\n" + "=" * 60)
print("Creating Agent with Custom Tools")
print("=" * 60)

custom_agent = Agent(
    client,
    model=model,
    instructions=(
        "You are an IT operations agent. "
        "Use the available tools to monitor and manage IT services. "
        "Always check service status before taking actions. "
        "Provide clear summaries of what you did."
    ),
    tools=[
        check_service_status,
        restart_service,
        get_all_services,
    ],
)

print("\n✅ Agent created with custom tools!")
print("   Tools: check_service_status, restart_service, get_all_services")
print("   Note: Tools run client-side (in this Python process)")

In [ ]:
# Step 4: Test the agent with custom tools
print("\n" + "=" * 60)
print("Testing Agent with Custom Tools")
print("=" * 60)

# Create a new session for custom tools
custom_session_id = custom_agent.create_session("test-custom-tools")
print(f"\n📝 Created session: {custom_session_id}")

# Test tasks
test_tasks = [
    "Check the status of all services",
    "Check the status of cache-service and restart it if it's degraded",
]

for task in test_tasks:
    print(f"\n{'=' * 60}")
    print(f"Task: {task}")
    print("=" * 60)
    
    cprint(f"User> {task}", "green")
    
    response = custom_agent.create_turn(
        messages=[{"role": "user", "content": task}],
        session_id=custom_session_id,
    )
    
    # AgentEventLogger shows tool calls!
    for log in AgentEventLogger().log(response):
        print(log, end="", flush=True)
    
    print()  # New line after each response

print(f"\n{'=' * 60}")
print("✅ All tests completed!")
print(f"{'=' * 60}")

---

## Summary

### What We Learned

1. **Built-in Tools**: Use `{"type": "web_search"}` for server-side tools
2. **Custom Tools**: Define Python functions with docstrings for client-side tools
3. **Agent Class**: Simplifies agent creation and tool management
4. **AgentEventLogger**: Shows tool calls visually (🤔 for reasoning, 🔧 for tools)
5. **Sessions**: Maintain conversation context across multiple turns

### Key Takeaways

✅ **Built-in tools** are easy to use - just specify the type  
✅ **Custom tools** are Python functions with docstrings  
✅ **Docstrings are critical** - the LLM reads them to understand tools  
✅ **AgentEventLogger** makes tool calls visible automatically  
✅ **Sessions** maintain context for multi-turn conversations  

### Tool Types Comparison

| Feature | Built-in Tools | Custom Tools |
|---------|---------------|--------------|
| **Location** | Server-side | Client-side |
| **Definition** | `{"type": "tool_name"}` | Python function |
| **Documentation** | Built-in | Docstrings |
| **Use Case** | Common operations (search, RAG) | Domain-specific logic |